# Motor Control

**Note: The wiring for this is very important.  Please double-check your connections before proceeding.**

In [1]:
# Import GPIO Libraries
import RPi.GPIO as GPIO
import time

In [14]:
GPIO.setmode(GPIO.BCM)

coil_A_1_pin = 18
coil_A_2_pin = 23
coil_B_1_pin = 24
coil_B_2_pin = 17

GPIO.setup(coil_A_1_pin, GPIO.OUT)
GPIO.setup(coil_A_2_pin, GPIO.OUT)
GPIO.setup(coil_B_1_pin, GPIO.OUT)
GPIO.setup(coil_B_2_pin, GPIO.OUT)

forward_seq = ['1010', '0110', '0101', '1001']
reverse_seq = list(forward_seq) # to copy the list
reverse_seq.reverse()


In [15]:
def forward(delay, steps):
  for i in range(steps):
    for step in forward_seq:
      set_step(step)
      time.sleep(delay)

def backwards(delay, steps):
  for i in range(steps):
    for step in reverse_seq:
      set_step(step)
      time.sleep(delay)


def set_step(step):
  GPIO.output(coil_A_1_pin, step[0] == '1')
  GPIO.output(coil_A_2_pin, step[1] == '1')
  GPIO.output(coil_B_1_pin, step[2] == '1')
  GPIO.output(coil_B_2_pin, step[3] == '1')



In [33]:
set_step('0000')
delay = 20 # Time Delay (ms)
steps = 60
forward(int(delay) / 1000.0, int(steps))

backwards(int(delay) / 1000.0, int(steps))

forward_seq = ['1010', '0110', '0101', '1001']
reverse_seq = list(forward_seq) # to copy the list
reverse_seq.reverse()

set_step('0000')
forward(int(delay) / 1000.0, int(steps))

backwards(int(delay) / 1000.0, int(steps))
set_step('0000')
